# Imports

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

from sklearn.model_selection import train_test_split

# Linear

In [ ]:
# output=XW^t + b

# X: Input tensor of shape (batch_size, in_features)
# W: Weight matrix of shape (out_features, in_features)
# b: Bias vector of shape (out_features,) (optional)

# Output shape: (batch_size, out_features)

# Input (batch_size x in_features)  →  Linear Layer  →  Output (batch_size x out_features)

In [3]:
layer = nn.Linear(4,2)

tensor = torch.randn(3,4)
output = layer(tensor)

In [7]:
tensor

tensor([[ 1.8879, -1.8185, -1.2606, -0.6749],
        [ 1.6505, -0.7917, -0.4010, -0.8471],
        [ 1.4576,  0.1465, -0.5967,  0.6151]])

In [6]:
layer.weight

Parameter containing:
tensor([[-0.2526,  0.1085, -0.4470, -0.0480],
        [-0.2012, -0.2811,  0.2503, -0.1745]], requires_grad=True)

In [8]:
layer.bias

Parameter containing:
tensor([-0.1905, -0.1388], requires_grad=True)

In [4]:
output

tensor([[-0.2687, -0.2053],
        [-0.4734, -0.2009],
        [-0.3056, -0.7300]], grad_fn=<AddmmBackward0>)

In [2]:
# Create synthetic regression dataset (1000 samples, 3 features)
np.random.seed(0)
X = np.random.rand(1000, 3) * [2000, 5, 50]
y = 50000 + (X[:, 0] * 100) + (X[:, 1] * 5000) - (X[:, 2] * 1000)
y = y.reshape(-1, 1)


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33,random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)   

In [3]:
class HousePriceMlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,1)
        )
    
    def forward(self, x):
        return self.model(x)
    
model = HousePriceMlp()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [4]:
for epoch in range(1000):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()

In [19]:
y_batch.dtype

torch.float32

In [5]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f"test loss: {test_loss.item()}")

test loss: 28489.3515625


In [11]:
loss.grad

/tmp/ipykernel_66008/3355736690.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  loss.grad


# Convolution Layer

In [ ]:
# output = (input_size + 2*padding - kernel_size)/stride + 1

# | Concept       | Meaning                       |
# | ------------- | ----------------------------- |
# | Kernel        | Small sliding filter          |
# | Feature map   | Output of the convolution     |
# | Stride        | Step size of the kernel       |
# | Padding       | Controls output size          |
# | in\_channels  | Input depth (e.g., 3 for RGB) |
# | out\_channels | Number of filters to learn    |

In [13]:
layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

x = torch.rand(1,3,32,32)
y = layer(x)

In [16]:
y.shape

torch.Size([1, 16, 32, 32])